In [ ]:
%%writefile processing.py
import sys

# Convert command line args into a map of args
args_iter = iter(sys.argv[1:])
args = dict(zip(args_iter, args_iter))


# Vehicle Routing Problem with Time Window
from flowty import Model, xsum
from or_datasets import vrp_rep
import os         
import pandas as pd

#accepts args as alias and instance
bunch = vrp_rep.fetch_vrp_rep(args['alias'], instance=args['instance'])
name, n, E, c, d, Q, t, a, b, x, y = bunch["instance"]

m = Model()

# one graph, it is identical for all vehicles
g = m.addGraph(obj=c, edges=E, source=0, sink=n - 1, L=1, U=n - 2, type="B")

# adds resources variables to the graph.
# travel time and customer tine windows
m.addResourceDisposable(
    graph=g, consumptionType="E", weight=t, boundsType="V", lb=a, ub=b, name="t"
)

# demand and capacity
m.addResourceDisposable(
    graph=g, consumptionType="V", weight=d, boundsType="V", lb=0, ub=Q, name="d"
)

# set partition constriants
for i in range(n)[1:-1]:
    m += xsum(x * 1 for x in g.vars if i == x.source) == 1

# packing set
for i in range(n)[1:-1]:
    m.addPackingSet([x for x in g.vars if i == x.source])

status = m.optimize()
#print(f"ObjectiveValue {m.objectiveValue}")

# get the variable values
for var in m.vars:
    if var.x > 0:
        print(f"{var.name} = {var.x}")

import math
import networkx
import matplotlib
import matplotlib.pyplot as plt

edges = [x.edge for x in g.vars if not math.isclose(x.x, 0, abs_tol=0.001)]
gx = networkx.DiGraph()
gx.add_nodes_from([i for i in range(n)])
gx.add_edges_from(edges)
pos = {i: (x[i], y[i]) for i in range(n)} # for lists of x,y coordinates
# pos = networkx.spring_layout(gx) # alternative layout
networkx.draw_networkx_nodes(gx, pos, nodelist=gx.nodes)
networkx.draw_networkx_labels(gx, pos, labels={i: i for i in gx.nodes})
networkx.draw_networkx_edges(gx, pos, nodelist=gx.edges)

#create objective_value dict 
objective_value = {"objective value":m.objectiveValue}
#create objective_value df 
objective_value_df = pd.DataFrame(objective_value.items())

#create variable_values dict 
variable_values = {}
for var in m.vars:
    if var.x > 0:
        variable_values[var.name] = var.x
#create variable_values df     
variable_values_df = pd.DataFrame(variable_values.items())    

try:
    os.makedirs("/opt/ml/processing/output/objective_value")
    os.makedirs("/opt/ml/processing/output/variable_values")
    os.makedirs("/opt/ml/processing/output/output/mygraph")
    print("Successfully created directories")
except Exception as e:
    # if the Processing call already creates these directories (or directory otherwise cannot be created)
    print(e)
    print("Could Not Make Directories")
    pass

# Save data locally
try:
    objective_value_df.to_csv("/opt/ml/processing/output/objective_value/objective_value.csv",index=False)
    variable_values_df.to_csv("/opt/ml/processing/output/variable_values/variable_values.csv",index=False)
    plt.savefig("/opt/ml/processing/output/mygraph/{}-{}-graph".format(args['alias'], args['instance']))
    print("Files Successfully Written")
except Exception as e:
    print("Could Not Write the Files")
    print(e)
    pass

print("Finished running processing job")

In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingOutput

#SET ecr_image 
ecr_image = <ACCOUNT_ID>.dkr.ecr.<REGION_ID>.amazonaws.com/<REPOSITORY_NAME>:<IMAGE_NAME>
#SET role_arn with the right permissions
role_arn = <ROLE_ARN>

script_processor = ScriptProcessor(command=['python3'],
                                   image_uri=ecr_image,
                                   role=role_arn,
                                   instance_count=1,
                                   instance_type='ml.m5.xlarge') 

In [ ]:
#SET code
processing_code = 'processing.py'
#SET alias
alias = 'solomon-1987-r1'
#SET instance_name
instance_name = 'R102_025'
#SET destination bucket for output data
output_bucket = <s3_bucket_name>

script_processor.run(code=processing_code,
                     arguments=['alias', alias,
                                'instance', instance_name
                               ],
                     outputs=[ProcessingOutput(source='/opt/ml/processing/output/objective_value', 
                                               destination='s3://{}/output/'.format(output_bucket)),
                              ProcessingOutput(source='/opt/ml/processing/output/variable_values', 
                                               destination='s3://{}/output/'.format(output_bucket)),
                              ProcessingOutput(source='/opt/ml/processing/output/mygraph', 
                                               destination='s3://{}/output/'.format(output_bucket)),
                             ],
                    )

In [ ]:
#output objective_value
import pandas as pd

objective_value = pd.read_csv('s3://' + output_bucket + '/output/objective_value.csv')
objective_value

In [ ]:
#output variable_values
variable_values = pd.read_csv('s3://' + output_bucket + '/output/variable_values.csv')
variable_values

In [ ]:
#display graph
import boto3
import botocore
from IPython import display

bucket_name = output_bucket
key = "output/" + alias +"-"+ instance_name +"-graph.png" 
file_name = alias +"-"+ instance_name +"-graph.png"

s3 = boto3.resource('s3')

try:
    s3.Bucket(bucket_name).download_file(key, file_name)
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

display.Image(file_name)